In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
print(os.listdir("../input"))
import subprocess

# subprocess.run('pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a', shell=True, check=True)

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score

from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

device=torch.device('cuda')


'''
一些参数的意义：
STAGE: 步骤，本代码需要在kernel上跑4次，第一次是STAGE=1,跑完以后导入kernel output（如果做validation大约需要7小时50分钟，如果不做需要6小时45分钟左右），然后将STAGE改成2,跑完以后更新kernel output，再更新STAGE=3。。。一直到STAGE=4
lr: learning_rate, default is 2e-5
I: 我做了一个5fold validation，I is the index of train/validation group, I in [0,1,2,3,4], 我们五个人可以正好每人负责一个fold
WARMUP: what fraction of t_total steps will be used for linear warmup, 不需要改变此参数
ES: 最后一个stage早停比例，设置次参数的意义是因为我不知道最后一个stage有没有过拟合
SEED: 不要改变次参数，保证seed不变可以让我们做fine tuning
batch_size: for kaggle kernel, this is best kept at 32, will run out of memory if 64，如果用v100可以适当提高一下看看结果有没有提高？
DO_VALIDATION: To save time, we can set it to false and only do validation in the STAGE=4

CV value for:
I=0: 0.9404
I=1: 0.9401
I=2: 0.9394
I=3: 0.9401
I=4: 0.9399

if your CV value for I is higher than the above, good job!!

'''


'''
可以微调的部分：
lr (learning_rate), you can try 2.5e-5, 1.5e-5 etc.
ES (last stage early stopping), you can try 0.9,0.8,0.75,0.7 etc. 
'''



lr = 2e-5
I = 1
STAGE = 4
WARMUP=0.05
ES=1
batch_size = 32

if ES>1:
    ES=1

DO_VALIDATION = False
kernel_filename = 'lr-reduction'

MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 2

# if STAGE==1:
#     FACTOR=1
#     MIN=0
#     MAX=22561
# if STAGE==2:
#     FACTOR=0.75
#     MIN=22562
#     MAX=45121
#     WARMUP=0
# elif STAGE==3:
#     FACTOR=0.5
#     MIN=0
#     MAX=22561
#     SEED*=2
#     WARMUP=0
# elif STAGE==4:
#     FACTOR=0.25
#     MIN=22562
#     MAX=int(45121*ES)
#     SEED*=2
#     WARMUP=0
#     DO_VALIDATION = True

lr=lr
FACTOR=1
DO_VALIDATION = True    
WARMUP=0

Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Input_dir = "../input"
WORK_DIR = "../working/"

TOXICITY_COLUMN = 'target'

package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam


BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'

convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin'
)

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'+'bert_config.json')
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)
    
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-12_H-768_A-12/'
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target']

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv"))
print('loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

#sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
#y_columns=['target']
# train_df = train_df.drop(['comment_text'],axis=1)
train_df['target']=(train_df['target']>=0.5).astype(float)


['crawl-300d-2M.pkl.zip', 'pytorch-pretrained-BERT', 'pytorch_model.bin', 'merges.txt', 'bert-pretrained-models', 'glove.840B.300d.pkl.zip', 'glove.840B.300d.pkl', 'gpt2_tokenizer.pickle', 'nvidiaapex', 'crawl-300d-2M.vec.zip', 'frpc', '.ipynb_checkpoints', 'glove.840B.300d.txt.zip', 'keras_bert', 'pytorch-pretrained-BERT-master', 'config.json', 'gpt2', 'crawl-300d-2M.pkl', 'uncased_L-12_H-768_A-12.zip', 'glove.840B.300d.txt', 'vocab.json', 'frpc.zip', 'jigsaw-unintended-bias-in-toxicity-classification', 'keras_bert.zip', 'crawl-300d-2M.vec']


/home/windywinter/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL
/home/windywinter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/windywinter/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/home/windywinter/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:4900: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/windywinter/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
/home/windywinter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size 

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /home/windywinter/JigSaw/input/bert-pretrained-models/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_

Loading TF weight bert/encoder/layer_5/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_5/attention/output/LayerNorm/gamma with shape [768]
Loading TF weight bert/encoder/layer_5/attention/output/dense/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/output/dense/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_5/attention/self/key/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/self/key/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_5/attention/self/query/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/self/query/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_5/attention/self/value/bias with shape [768]
Loading TF weight bert/encoder/layer_5/attention/self/value/kernel with shape [768, 768]
Loading TF weight bert/encoder/layer_5/intermediate/dense/bias with shape [3072]
Loading TF weight bert/encoder/layer_5/intermediate/dense/

loaded 1804874 records


In [ ]:

# #np.save('sequences_{}_{}'.format(num_to_load, MAX_SEQUENCE_LENGTH), sequences)
# print('loading sequences')
# sequences = np.load('../input/sequences320/sequences_320.npy')[:,:MAX_SEQUENCE_LENGTH]

# with open('../input/sequences320/skf_5_splits.pkl', 'rb') as f:
#     splits = pickle.load(f)

sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df = train_df.drop(['comment_text'],axis=1)

print('generating x train and valid')

train_size = int(len(train_df)*0.95)

X = sequences[:train_size]
X_val = sequences[train_size:]
test_df = train_df[train_size:]


weights = np.ones(len(train_df)) / 4
weights += (train_df[identity_columns].fillna(0).values >= 0.5).sum(axis=1).astype(bool).astype(np.int) / 4
weights += (( (train_df['target'].values >= 0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values < 0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
weights += (( (train_df['target'].values < 0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values >= 0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

loss_weight = 1.0 / weights.mean()

Y = np.vstack([(train_df['target'].values >= 0.5).astype(np.int), weights]).T
Y_aux = train_df[['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']]
Y = np.hstack((Y, Y_aux))

print('generating y train and valid')
y = Y[:train_size]
y_val = Y[train_size:]

del train_df, sequences, Y, Y_aux
gc.collect()


train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))
del X, y, tokenizer
gc.collect()


A Jupyter Widget


40661
generating x train and valid
generating y train and valid


7

0

In [ ]:

def custom_loss(y_pred, y_true):
    bce_loss_1 = F.binary_cross_entropy_with_logits(y_pred[:,:1].reshape(-1), y_true[:,:1].reshape(-1), weight=y_true[:,1:2].reshape(-1))   
    N = y_true.shape[1]
    custom_loss = bce_loss_1
    for i in range(N-2):
        custom_loss += F.binary_cross_entropy_with_logits(y_pred[:,1+i], y_true[:,2+i])
    return custom_loss

output_model_file = 'full_train_torch_bert_epoch_2.bin'
accumulation_steps=1
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

print('Initiating model for training, output model filename is {}'.format(output_model_file))

model = BertForSequenceClassification.from_pretrained("../working", cache_dir=None, num_labels=7)
model.zero_grad()

# if STAGE == 1:
#     model = BertForSequenceClassification.from_pretrained("../working", cache_dir=None, num_labels=7)
#     model.zero_grad()
# else:
#     model = BertForSequenceClassification(bert_config, num_labels=7)
#     model.load_state_dict(torch.load("../input/{}/fold_{}_lr_reduction_stage_{}.bin".format(kernel_filename, I, STAGE-1)))

model = model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    

# num_train_optimization_steps decrease proportionally to lr
num_train_optimization_steps = int(FACTOR*2*EPOCHS * len(train_dataset.tensors[0]) /batch_size/accumulation_steps)
print('num_train_optimization_steps is {}'.format(num_train_optimization_steps))

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=WARMUP,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    gc.collect()
    
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    for i,(x_batch, y_batch) in tk0:
        
        optimizer.zero_grad()
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
            #loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        loss = custom_loss(y_pred, y_batch.to(device))
            
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)

torch.save(model.state_dict(), output_model_file)
del train_loader
gc.collect()


if DO_VALIDATION:
    print('Starting validation')
    model = BertForSequenceClassification(bert_config, num_labels=7)
    model.load_state_dict(torch.load(output_model_file))
    model.to(device)
    for param in model.parameters():
        param.requires_grad=False
    model.eval()
    valid_preds = np.zeros((len(X_val)))
    valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

    tk0 = tqdm_notebook(valid_loader)
    for i,(x_batch,)  in enumerate(tk0):
        pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()


    def calculate_overall_auc(df, model_name):
        true_labels = df[TOXICITY_COLUMN]>0.5
        predicted_labels = df[model_name]
        return metrics.roc_auc_score(true_labels, predicted_labels)

    def power_mean(series, p):
        total = sum(np.power(series, p))
        return np.power(total / len(series), 1 / p)

    def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
        bias_score = np.average([
            power_mean(bias_df[SUBGROUP_AUC], POWER),
            power_mean(bias_df[BPSN_AUC], POWER),
            power_mean(bias_df[BNSP_AUC], POWER)
        ])
        return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)

    SUBGROUP_AUC = 'subgroup_auc'
    BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
    BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

    def compute_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except ValueError:
            return np.nan

    def compute_subgroup_auc(df, subgroup, label, model_name):
        subgroup_examples = df[df[subgroup]>0.5]
        return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

    def compute_bpsn_auc(df, subgroup, label, model_name):
        """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
        subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
        non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
        examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
        return compute_auc(examples[label]>0.5, examples[model_name])

    def compute_bnsp_auc(df, subgroup, label, model_name):
        """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
        subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
        non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
        examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
        return compute_auc(examples[label]>0.5, examples[model_name])

    def compute_bias_metrics_for_model(dataset,
                                       subgroups,
                                       model,
                                       label_col,
                                       include_asegs=False):
        """Computes per-subgroup metrics for all subgroups and one model."""
        records = []
        for subgroup in subgroups:
            record = {
                'subgroup': subgroup,
                'subgroup_size': len(dataset[dataset[subgroup]>0.5])
            }
            record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
            record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
            record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
            records.append(record)
        return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


    MODEL_NAME = 'model1'
    test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
    TOXICITY_COLUMN = 'target'
    bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
    bias_metrics_df
    print(get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME)))


    test_df.to_csv('test_df.csv')
    bias_metrics_df.to_csv('bias_metrics_df.csv')



Initiating model for training, output model filename is full_train_torch_bert_epoch_2.bin
num_train_optimization_steps is 214328


A Jupyter Widget

428

A Jupyter Widget